In [6]:
import pypyodbc as odbc #!pip install pypyodbc
import pandas as pd
import time
import os

print(odbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client RDA 11.0']


#### Needed Inputs:

In [7]:
driverName = "SQL Server" 
serverName = "DESKTOP-UVMKETP\SQLEXPRESS"
dataBaseName = "SQL_PowerBI_Test"
username = "jvenncpe"
pwd = "#Juv1994*"

tableName = "product_category_name_translation"
csv_FilePaTH = r"G:\00 Work Files\02 Work Trainings & Apps\00 Project Portfolio\Ecommerce_Amazon (Python, SQL, PowerBI)\CSV Files\Amazon_Olist"
csv_FileName = r"product_category_name_translation.csv" #To be used for column name reference in SQL Table
csv_FolderName = rf"{tableName}"

### Custom Functions

In [8]:
def updateSQL():
    try:
        conn.commit()
        return print("SQL Updated Successfully!")
        
    except Exception as e:
        return print(f"Error occurred: {e}")

def SQL_createTable_columnName(column):
    column_names = []
    for x in column:
        x = f'[{x}] VARCHAR(MAX)'
        column_names.append(x)
    return ', '.join(column_names)  # Join the column names with commas

def SQL_createTable_query(tableName,tableColumns):
    createTable_query = f"\
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{tableName}')\
    BEGIN\
        CREATE TABLE {dataBaseName}.dbo.{tableName}({tableColumns})\
    END".strip()
    return createTable_query

def bulk_insert(datapath_file, SQL_Table):
    insertBulk_query = f"""
        BULK INSERT {SQL_Table}
        FROM '{datapath_file}'
        WITH
        (
            FIELDQUOTE = '"',
            FIRSTROW = 2,
            FIELDTERMINATOR = ',', 
            ROWTERMINATOR = '\n',  
            TABLOCK
        ) 
    """.strip()
    return insertBulk_query

#### Adhoc SQL Configuration

In [9]:
connection_string = f"""
    DRIVER={{{driverName}}};
    SERVER={serverName};
    DATABASE={dataBaseName};
    Trust_Connection=yes;
    UID={username};
    PWD={pwd};
"""
try:
    conn = odbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

except Exception as e:
    print(f"Error: {e}")


Connection successful!


#### Main Function

In [10]:
if __name__ == '__main__':
    csv_filePaTH_REF = os.path.join(csv_FilePaTH,csv_FolderName,csv_FileName)   
    csv_FoldderPath = os.path.join(csv_FilePaTH, csv_FolderName)
    
    df_productDetails = pd.read_csv(csv_filePaTH_REF, encoding='latin1', na_values=[""])
    
    for columnName in df_productDetails.columns:
        columnName_After = columnName.replace(".","")
        columnName_After = columnName_After.replace(" ","_")
        df_productDetails.rename(columns={columnName: columnName_After}, inplace=True)
    
    cursor = conn.cursor()
    tableColumns = SQL_createTable_columnName(df_productDetails.columns)
    
    try:
        with cursor:
            cursor.execute(SQL_createTable_query(tableName,tableColumns))
            updateSQL()
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')
    
    data_files = os.listdir(csv_FoldderPath)
    SQL_Table = f'{dataBaseName}.dbo.{tableName}'
    
    cursor = conn.cursor()
    try:
        with cursor:
            print(f"Target SQL Table: {SQL_Table}")
            print(f"---------------------------")
            print(f"Files to Upload:")
            
            for data_file in data_files:
                if data_file.endswith('.csv'):
                    cursor.execute(bulk_insert(os.path.join(csv_FoldderPath, data_file), SQL_Table))
                    print(data_file)
                    
            print(f"---------------------------")
            updateSQL()
            
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')

SQL Updated Successfully!
Target SQL Table: SQL_PowerBI_Test.dbo.product_category_name_translation
---------------------------
Files to Upload:
product_category_name_translation.csv
---------------------------
SQL Updated Successfully!


In [11]:
df_productDetails.head(5)

,ï»¿product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
